# 🌟 LangChain 入门与实战教程

---

## 🧠 什么是 LangChain？

**LangChain** 是一个用于构建基于大语言模型（LLM）的应用框架。  
它的核心目标是 ——  
> 让我们能像“搭积木”一样，把 LLM、Prompt、Memory、Tool、Database 等模块**组合成一个完整系统**。

换句话说，LangChain 让 AI 不再只是“聊天”，  
而是能**记忆、推理、检索、调用工具**、执行完整逻辑。

---

## 🧩 LangChain 的核心组件

| 模块 | 作用 | 示例 |
|------|------|------|
| **Prompt** | 设定提示模板 | “请分析以下文本情感：{text}” |
| **LLM** | 调用大模型（如 OpenAI、Qwen、Claude） | `ChatOpenAI()` |
| **Parser** | 解析模型输出为结构化结果 | `StrOutputParser()`、`with_structured_output()` |
| **Chain (LCEL)** | 用管道符把各模块串联起来 | Prompt - LLM -  Parser
| **Memory** | 保存历史对话，实现上下文记忆 | `RunnableWithMessageHistory()` |
| **Tool** | 工具调用，例如计算、查天气 | `@tool` |
| **RAG** | 检索增强问答，结合向量数据库 | FAISS / Chroma |
| **Fallback** | 模型或链出错时的自动回退 | `.with_fallbacks()` |

---

## ⚙️ LangChain Expression Language（LCEL）

LangChain 2.0 引入了新的“表达式语言” —— **LCEL**，  
用一个符号 `|` 串联整个流程，让链式逻辑更清晰：

```python
ChatPromptTemplate.from_template("问题：{question}") 
| llm 
| StrOutputParser()


# 1. LCEL 最简单的问答链

In [ ]:
"""
最简单的 LCEL 问答
"""

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 定义模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://www.dmxapi.cn/v1",
    api_key="xxx", # 替换为你的 API Key 
    temperature=0)

# 构建链：Prompt -> LLM -> 解析文本
qa_chain = (
    ChatPromptTemplate.from_template("用户问题：{question}")
    | llm
    | StrOutputParser()
)

# 执行问答
if __name__ == "__main__":
    result = qa_chain.invoke({"question": "中国的首都是哪？"})
    print("AI答复：", result)


: 

# 2. 结构化输出（Pydantic Schema）

In [ ]:

"""
LCEL 格式化输出案例：
输入一段文本 → 返回结构化结果（情感分类 + 主题 + 总结）
"""

from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 1. 定义输出 Schema
class AnalysisResult(BaseModel):
    sentiment: Literal["积极", "中性", "消极"] = Field(..., description="文本的情感极性")
    topic: str = Field(..., description="主要主题")
    summary: str = Field(..., description="简要总结")

# 2. 定义模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
    temperature=0)

# 3. 构建 LCEL 链：Prompt | LLM | StructuredOutput
analysis_chain = (
    ChatPromptTemplate.from_template("请分析以下文本，并提取关键信息：{text}")
    | llm.with_structured_output(AnalysisResult)
)

# 4. 执行
if __name__ == "__main__":
    result: AnalysisResult = analysis_chain.invoke(
        {"text": "今天股市大涨，我对未来的投资充满信心！"}
    )
    print(result.dict())


{'sentiment': '积极', 'topic': '股市表现与投资信心', 'summary': '今天股市表现良好，作者对未来的投资持乐观态度。'}


/tmp/ipykernel_948/3080188056.py:35: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(result.dict())


# 3. 历史记忆与消息占位符

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# 1. 定义 Prompt，包含历史消息占位符
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个友好的中文助理。"),
        MessagesPlaceholder("history"),   # 占位符：历史对话会注入这里
        ("human", "{question}")           # 当前用户问题
    ]
)

# 2. 构造历史消息
history = [
    HumanMessage(content="1+1 = ?"),
    AIMessage(content="1+1 = 2")
]

# 3. 格式化 Prompt，把历史消息 + 新问题合并
prompt_value = prompt.format_messages(history=history, question="我刚才问的问题是什么？")
print(prompt_value)

# 4. 定义大模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
    temperature=0)

# 5. 执行调用
response = llm.invoke(prompt_value)
print("AI答复：", response.content)


[SystemMessage(content='你是一个友好的中文助理。', additional_kwargs={}, response_metadata={}), HumanMessage(content='1+1 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='1+1 = 2', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚才问的问题是什么？', additional_kwargs={}, response_metadata={})]
AI答复： 你刚才问的问题是“1+1 = ?”。


# 4. 工具调用（Tool Calling）

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage, AIMessage

# 定义一个工具：计算两个数的和
@tool
def add(a: int, b: int) -> int:
    """计算两个整数的和"""
    print("tool ....")
    return a + b

# 模型绑定工具
llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
    temperature=0)

llm_tools = llm.bind_tools([add])

# 用户提问
user_input = "请帮我算一下 12 + 30 等于多少，如果需要可以用工具。"
# 第一次调用模型 → 可能触发工具调用
ai_msg = llm_tools.invoke(user_input)

# 如果模型触发了工具调用
if ai_msg.tool_calls:
    for call in ai_msg.tool_calls:
        if call["name"] == "add":  # 执行工具
            result = add.invoke(call["args"])
            tool_msg = ToolMessage(content=str(result), name=call["name"], tool_call_id=call["id"])
            # 再次调用模型 → 让它基于工具结果生成自然语言回答
            final_ai = llm_tools.invoke([("human", user_input), ai_msg, tool_msg])
            print("调用工具最终回答:", final_ai.content)
else:
    # 模型可能直接回答
    print("最终回答:", ai_msg.content)

tool ....
调用工具最终回答: 12 + 30 等于 42。


# 5. 流式输出（Streaming）

In [ ]:
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

stream_llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


_ = stream_llm.invoke("请用三句话介绍LangChain的核心功能。")


LangChain是一种用于构建智能应用的框架，特别关注自然语言处理和生成。它通过集成大型语言模型（LLMs）和各种数据源，支持复杂的对话管理和推理能力。该框架还提供了工具和接口，方便开发者构建和部署多样化的语言应用。

# 6. 批处理与异步并发

In [ ]:
import asyncio

# 模型绑定工具
llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
    temperature=0)

prompt = ChatPromptTemplate.from_template("一句话回答：{q}")
chain = prompt | llm | StrOutputParser()
qs = [{"q": q} for q in ["什么是RAG？", "什么是Agent？", "什么是LangGraph？"]]

# 同步
print(chain.batch(qs))

# 异步 py文件
# async def run():
#     rets = await chain.abatch(qs)
#     print(rets)
# asyncio.run(run())

# 异步 notebook文件
rets = await chain.abatch(qs)
print(rets)

['RAG（Retrieval-Augmented Generation）是一种结合信息检索和生成模型的技术，用于提高自然语言处理任务的性能。', 'Agent是能够自主感知环境并采取行动以实现特定目标的实体。', 'LangGraph是一种用于自然语言处理的图结构，旨在通过图形化方式表示和处理语言数据及其关系。']
['RAG（Retrieval-Augmented Generation）是一种结合信息检索和生成模型的技术，用于提高自然语言处理任务的性能。', 'Agent是能够自主感知环境并采取行动以实现特定目标的实体。', 'LangGraph是一种用于自然语言处理的图结构，旨在通过图形化方式表示和处理语言数据及其关系。']


# 7. LCEL 分支与并行

In [20]:
from langchain_core.runnables import RunnableBranch, RunnableParallel

summ = ChatPromptTemplate.from_template("用中文总结：{text}")
bullets = ChatPromptTemplate.from_template("列出三条要点：{text}")

parallel = RunnableParallel(
    summary=summ | llm | StrOutputParser(),
    points=bullets | llm | StrOutputParser(),
)
print(parallel.invoke({"text": "LangChain 是一个模块化的 LLM 应用框架，用于构建复杂对话、RAG、Agent 等系统。"}))


{'summary': 'LangChain 是一个模块化的 LLM 应用框架，旨在构建复杂的对话系统、检索增强生成（RAG）和智能代理（Agent）等应用。', 'points': '1. **模块化设计**：LangChain 提供了灵活的模块化架构，使开发者能够根据需求组合不同的组件，构建复杂的对话系统、检索增强生成（RAG）和智能代理（Agent）等应用。\n\n2. **支持多种功能**：该框架支持多种功能，包括自然语言处理、信息检索、上下文管理和任务执行，帮助开发者快速实现复杂的应用场景。\n\n3. **易于集成**：LangChain 可以与多种外部数据源和 API 集成，增强系统的能力和灵活性，适用于各种行业和应用需求。'}


# 8. 可持续记忆（RunnableWithMessageHistory）

In [21]:
# =====================================================
# 可持续记忆：RunnableWithMessageHistory
# =====================================================

from langchain_core.chat_history import InMemoryChatMessageHistory  # 内存型消息记录
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser


# 定义全局的“会话存储”，用来保存每个 session 的聊天历史
#    （真实项目中可改为 Redis、SQLite 等）
store = {}

def get_session_history(session_id: str):
    """
    根据 session_id 获取对应的历史消息对象。
    如果不存在则创建一个新的 InMemoryChatMessageHistory。
    """
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


# 定义 Prompt 模板
#     - system: 给模型设定角色
#     - MessagesPlaceholder: 历史消息将注入这里
#     - human: 当前用户输入
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个友好的中文助理，会根据上下文回答问题。"),
    MessagesPlaceholder("history"),
    ("human", "{question}")
])


#构建基本链：Prompt → LLM → 输出解析
memory_chain = prompt | llm | StrOutputParser()

# -----------------------------------------------------
# 将链包装为支持记忆的版本
with_history = RunnableWithMessageHistory(
    memory_chain,              # 原始链
    get_session_history,       # 获取历史函数
    input_messages_key="question",  # 对应 prompt 输入的 key
    history_messages_key="history", # 对应 MessagesPlaceholder 的变量名
)

# -----------------------------------------------------
# 模拟一个会话，用 session_id 区分不同用户
cfg = {"configurable": {"session_id": "user-001"}}

# 第一次提问：告诉模型“我叫张三”
print("用户：我叫张三。")
print("AI：", with_history.invoke({"question": "我叫张三。"}, cfg))

# 第二次提问：让模型回忆前面的对话
print("\n 用户：我叫什么？")
print("AI：", with_history.invoke({"question": "我叫什么？"}, cfg))



用户：我叫张三。
AI： 你好，张三！很高兴认识你。有什么我可以帮助你的吗？

 用户：我叫什么？
AI： 你叫张三。有什么想聊的或者需要帮助的呢？


# 9. 简单RAG 

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

docs = [
    "LangChain 是一个用于构建大语言模型应用的框架。",
    "LangGraph 用于多Agent协作与分支逻辑建模。",
    "LangSmith 用于可观测性与评估。",
    "LangFlow 是可视化拖拽式构建器。"
]
splits = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=10).create_documents(docs)

emb = OpenAIEmbeddings(
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
    model="text-embedding-3-small",
)
vs = FAISS.from_documents(splits, emb)

def retrieve(q, k=3): return vs.similarity_search(q, k=k)
rag_prompt = ChatPromptTemplate.from_template("已知内容：\n{context}\n---\n回答问题：{question}")
def format_docs(ds): return "\n".join([d.page_content for d in ds])

# 定义一个问答函数，先检索，再问答
def rag_chain(question: str):
    # Step 1: 检索相似文档
    docs = retrieve(question)
    print('docs:' , docs)
    context = format_docs(docs)
   

    # Step 2: 生成 Prompt
    prompt_text = rag_prompt.format(context=context, question=question)

    # Step 3: 调用模型
    response = llm.invoke(prompt_text)

    # Step 4: 提取文本结果
    return response.content
    
print(rag_chain("LangGraph 是做什么的？"))



docs: [Document(id='69c1dd1f-bce0-419c-b13b-0577d2eee490', metadata={}, page_content='LangGraph 用于多Agent协作与分支逻辑建模。'), Document(id='8cff42bf-2fe9-418c-99c9-7f22ccb9e4b0', metadata={}, page_content='LangChain 是一个用于构建大语言模型应用的框架。'), Document(id='f7eb9fa8-9ec6-4962-a2a3-dc8aa2f74121', metadata={}, page_content='LangFlow 是可视化拖拽式构建器。')]
LangGraph 是用于多Agent协作与分支逻辑建模的工具。


# 10. 容错与回退机制（Fallback）

In [ ]:
"""
当主模型超时 / 报错 / 不稳定 / 价格太贵时，
系统可以自动「切换」到另一个备用模型继续回答，
而不会让用户看到 “请求失败” 的尴尬场面。
"""

fallback_llm = ChatOpenAI(
    model="gpt-4o-mini-legacy",
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
)

robust_chain = (
    ChatPromptTemplate.from_template("回答：{q}")
    | llm | StrOutputParser()
).with_fallbacks([
    ChatPromptTemplate.from_template("回答精简：{q}") | fallback_llm | StrOutputParser()
])

print(robust_chain.invoke({"q": "简述 LangSmith 的作用"}))


LangSmith 是一个专注于自然语言处理（NLP）和机器学习的工具或平台，旨在帮助开发者和研究人员更高效地构建、训练和部署语言模型。它的主要作用包括：

1. **模型训练**：提供用户友好的界面和工具，简化语言模型的训练过程，使开发者能够更快地实现模型的训练和优化。

2. **数据管理**：帮助用户管理和处理训练数据，包括数据清洗、标注和增强等功能，以提高模型的性能。

3. **评估与调优**：提供评估工具，帮助用户分析模型的表现，并进行必要的调优，以达到更好的效果。

4. **集成与部署**：支持将训练好的模型集成到应用程序中，并提供部署方案，使得模型能够在实际环境中运行。

5. **社区与支持**：通过社区支持和文档，帮助用户解决在使用过程中遇到的问题，促进知识分享和合作。

总的来说，LangSmith 旨在降低自然语言处理项目的技术门槛，提高开发效率。
